In [1]:
import sys
import os
# Adjust the path if needed based on where your src directory is located relative to this notebook
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from src.utils.openFoamUtils import *
from src.utils.data_utils import *
from src.models.tbnn import *
from src.config import OPENFOAM_DATA_DIR

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

import random

def set_seed(seed=42):
    """Set seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed set to {seed}")

# Set the seed
set_seed(42)  # You can choose any integer value

Using device: cuda
Random seed set to 42


In [2]:
sequence_length = 25
stride = 10
invariants = ['I1_1', 'I1_3', 'I1_5', 'I1_7', 'I1_13', 'I1_29', 'I1_33', 'q2', 'q3', 'q4']
target = 'bij'
#Create Dataset
X_stacked_846, Y_stacked_846, TB_stacked_846,_,_,y_values_846 = read_dataset(846, sequence_length,stride,invariants=invariants,target=target,filter=5)
X_stacked_1155, Y_stacked_1155, TB_stacked_1155,_,_,y_values_1155 = read_dataset(1155,sequence_length,stride,invariants=invariants,target=target,filter=5)
X_stacked_1475, Y_stacked_1475, TB_stacked_1475,_,_,y_values_1475 = read_dataset(1475,sequence_length,stride,invariants=invariants,target=target,filter=5)

# Exclude the first spatial point (index 0) from X, Y, and TB
X_training = torch.cat([X_stacked_846, X_stacked_1475])[:,:,:,:].to(device)
Y_training = torch.cat([Y_stacked_846, Y_stacked_1475])[:,:,:].to(device)
TB_training = torch.cat([TB_stacked_846, TB_stacked_1475])[:,:,:,:].to(device)

X_validation = X_stacked_1155[:,:,:,:].to(device)
Y_validation = Y_stacked_1155[:,:,:].to(device)
TB_validation = TB_stacked_1155[:,:,:,:].to(device)

In [3]:
# Load both configuration and weights
criterion = nn.MSELoss()
checkpoint = torch.load('best_model_tbnn_complete.pth')
hyperparameters = checkpoint['hyperparameters']

# Create model with the same architecture
model = LSTM_TBNN(**hyperparameters).to(device)

# Load the weights
model.load_state_dict(checkpoint['model_state_dict'])

x_train_test = X_validation
y_train_test = Y_validation
TB_train_test = TB_validation
#load best model
model.load_state_dict(torch.load('best_model_tbnn_20.pth'))
model.eval()
val_losses = []
y_pred_list = []
rel_errors = []
with torch.no_grad():
    for t in range(x_train_test.shape[0]):
        x_val, y_val = x_train_test[t], y_train_test[t]
        TB_val = TB_train_test[t]
        x_val, y_val, TB_val = x_val, y_val, TB_val
        ypred_val = model(x_val,TB_val)
        #P_val = prod_term_train_test[t]
        #x_val, y_val, TB_val, grad_u_val, P_val = x_val, y_val, TB_val, grad_u_val, P_val
        #s_pred_val = model(x_val, TB_val, grad_u_val)
        #ypred_val = s_pred_val * P_val.view(-1,1) 
        y_pred_list.append(ypred_val.cpu().detach().numpy())
        loss = criterion(ypred_val, y_val)
        #relative error
        # Calculate relative error
        relative_error = np.abs(ypred_val.cpu().detach().numpy() - y_val.cpu().detach().numpy()) / np.abs(y_val.cpu().detach().numpy())
        rel_errors.append(np.mean(relative_error))
        val_losses.append(loss.item())
    #print(f"Loss: {loss.item():.6f}")
# Calculate average loss
avg_loss = np.mean(val_losses)
print(f"Average validation loss: {avg_loss:.6f}")
y_pred_list = np.array(y_pred_list)



Average validation loss: 0.001765


In [4]:
Re = 1155
base_path = OPENFOAM_DATA_DIR
kFrozenProp_directory = f"{base_path}/tbnn/Re{Re}"
kFrozen_directory = f"{base_path}/kFrozen/Re{Re}"
bij_predictions = np.zeros((700,160,3,3))
bij_predictions[:,:,0,0] = y_pred_list[:,:,0]
bij_predictions[:,:,0,1] = bij_predictions[:,:,1,0] = y_pred_list[:,:,1]
bij_predictions[:,:,1,1] = y_pred_list[:,:,2]
bij_predictions[:,:,2,2] = y_pred_list[:,:,3]

list_and_write_reynolds_stress(kFrozen_directory, kFrozenProp_directory+"/reynoldStress", bij_predictions)

Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.01/bijDelta
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.02/bijDelta
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.03/bijDelta
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.04/bijDelta
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.05/bijDelta
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_pre

In [5]:
sequence_length = 25
stride = 10
invariants = ['I1_1', 'I1_3', 'I1_5', 'I1_7', 'I1_13', 'I1_29', 'I1_33', 'q2', 'q3', 'q4','krans']
target = 'kDeficit'
#Create Dataset
X_stacked_846, Y_stacked_846, TB_stacked_846,_, gradU_stacked_846,y_values_846 = read_dataset(846, sequence_length,stride,invariants=invariants,target=target,filter=5,gradU_column= 'gradU_0_1')
X_stacked_1155, Y_stacked_1155, TB_stacked_1155,_, gradU_stacked_1155,y_values_1155 = read_dataset(1155,sequence_length,stride,invariants=invariants,target=target,filter=5,gradU_column= 'gradU_0_1')
X_stacked_1475, Y_stacked_1475, TB_stacked_1475,_, gradU_stacked_1475,y_values_1475 = read_dataset(1475,sequence_length,stride,invariants=invariants,target=target,filter=5,gradU_column= 'gradU_0_1')

# Exclude the first spatial point (index 0) from X, Y, and TB
X_training = torch.cat([X_stacked_846, X_stacked_1475])[:,:,:,:].to(device)
Y_training = torch.cat([Y_stacked_846, Y_stacked_1475])[:,:,:].to(device)
TB_training = torch.cat([TB_stacked_846, TB_stacked_1475])[:,:,:,:].to(device)
gradU_training = torch.cat([gradU_stacked_846, gradU_stacked_1475])[:,:,:].to(device)

X_validation = X_stacked_1155[:,:,:,:].to(device)
Y_validation = Y_stacked_1155[:,:,:].to(device)
TB_validation = TB_stacked_1155[:,:,:,:].to(device)
gradU_validation = gradU_stacked_1155[:,:,:].to(device)

class SafeRelativeMSELoss(nn.Module):
    def __init__(self, min_target=0.1):
        super(SafeRelativeMSELoss, self).__init__()
        self.min_target = min_target  # avoid division by very small values

    def forward(self, y_pred, y_true):
        denominator = torch.clamp(y_true, min=self.min_target)  # clamp to min_target
        relative_error = (y_pred - y_true) / denominator
        return torch.mean(relative_error ** 2)

class LogCoshLoss(nn.Module):
    def forward(self, y_pred, y_true):
        loss = torch.log(torch.cosh(y_pred - y_true))
        return torch.mean(loss)

criterion = SafeRelativeMSELoss() #LogCoshLoss() #SafeRelativeMSELoss()

checkpoint = torch.load('best_model_k_tbnn_w_hparm.pth')
hyperparameters = checkpoint['hyperparameters']



# Create model with the same architecture
model = LSTM_TBNN_R(**hyperparameters).to(device)
# Load the weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
val_losses = []
y_pred_list = []
rel_errors = []
with torch.no_grad():
    for t in range(X_validation.shape[0]):
        x_batch = X_validation[t]
        y_batch = Y_validation[t]
        TB_batch = TB_validation[t]
        gradU_batch = gradU_validation[t]
        
        # Prediction step
        y_pred = model(x_batch, TB_batch, gradU_batch)
        
        # Store predictions
        y_pred_list.append(y_pred.cpu().detach().numpy())
        
        # Calculate loss
        loss = criterion(y_pred, y_batch)
        val_losses.append(loss.item())
        
        # Calculate relative error
        relative_error = np.abs(y_pred.cpu().detach().numpy() - y_batch.cpu().detach().numpy()) / np.abs(y_batch.cpu().detach().numpy())
        rel_errors.append(np.mean(relative_error))
        
# Calculate average loss
avg_loss = np.mean(val_losses)
print(f"Average validation loss: {avg_loss:.6f}")
y_pred_list = np.array(y_pred_list)

Average validation loss: 0.009348


In [6]:
kDeficit_pred = y_pred_list
list_and_write_kDeficit(kFrozen_directory,kFrozenProp_directory+"/reynoldStress",kDeficit_pred)

Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.01/kDeficit
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.02/kDeficit
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.03/kDeficit
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.04/kDeficit
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_prediction/data/openfoam/tbnn/Re1155/reynoldStress/0.05/kDeficit
Reynolds stress file has been written to /home/fredrik/Documents/Tensor_basis_neural_networks_for_unsteady_flow_pre